In [33]:
import pandas as pd
import numpy as np
import math
import itertools
import re
import scipy.stats as sp


class arayi(object):
    
    def removeCols(self, csvPath):
        df=pd.read_csv(csvPath)
        dropCols = range(0,15)
        dropCols += range(16,29)
        df.drop(df.columns[dropCols],axis=1,inplace=True)
        return df

    def calcFAC(self,df):
        facDict={}
        for idx, row in df.iterrows():
            ## Build dictionary with track keys
            for cell in row[1:11]:
                if cell.split("/")[5] not in facDict:
                    facDict[cell.split("/")[5]]={"familiar":0,"listen":0,"sing":0}

            ## Count number of times track is familiar
            for loc, famil in enumerate(row[11:21]):
                if famil == "Y":
                    facDict[row[loc+1].split("/")[5]]["familiar"]+=1

            # Count number of times a song was preferred to listen
            for loc, listen in enumerate(row[28:33]):
                if listen == 1:
                    header="Input.link%sa" % str(loc+1)
                    facDict[row[header].split("/")[5]]["listen"]+=1
                else:
                    header="Input.link%sb" % str(loc+1)
                    facDict[row[header].split("/")[5]]["listen"]+=1
            
            # Count number of times a song was preferred to be sung
            for loc, sing in enumerate(row[51:56]):
                if sing == 1:
                    header="Input.link%sa" % str(loc+1)
                    facDict[row[header].split("/")[5]]["sing"]+=1
                else:
                    header="Input.link%sb" % str(loc+1)
                    facDict[row[header].split("/")[5]]["sing"]+=1

        return facDict

    def calcAHP(self,df):
        ahpDict={"globals":{"famil":{"genre":[],"repro":[],"listen":[]},"genre":{"listen":[],"repro":[]},"listen":{"repro":[]}},"locals":{"famil":{"Yes":[]},"listen":{"Low":{"Mid":[],"High":[]},"Mid":{"High":[]}},"repro":{"Easy":{"Mid":[],"Hard":[]},"Mid":{"Hard":[]}},"genre":{"Rock":{"Pop":[],"Alt":[],"Country":[],"Rap":[]},"Pop":{"Alt":[],"Country":[],"Rap":[]},"Alt":{"Country":[],"Rap":[]},"Country":{"Rap":[]}}}}

        for idx, row in df.iterrows():

            ## Globals
            for ind,glob in enumerate(row[22:28]):
                header = df.columns[ind+22].split("_")[1]
                if glob == 1:
                    glob=0.0
                else:
                    glob=float(glob)

                if np.sign(glob) == 1:
                    try:
                        ahpDict["globals"][header.split("-")[1]][header.split("-")[0]].append(glob)
                    except:
                        ahpDict["globals"][header.split("-")[0]][header.split("-")[1]].append(1/glob)
                elif np.sign(glob) == -1:
                    try:
                        ahpDict["globals"][header.split("-")[0]][header.split("-")[1]].append(abs(glob))
                    except:
                        ahpDict["globals"][header.split("-")[1]][header.split("-")[0]].append(abs(1/glob))
                else:
                    try:
                        ahpDict["globals"][header.split("-")[0]][header.split("-")[1]].append(1.0)
                    except:
                        ahpDict["globals"][header.split("-")[1]][header.split("-")[0]].append(1.0)

            ## Locals
#             print df.columns[33:50]
            # Familiar
            famLoc = float(row[33])
            if np.sign(famLoc) < 0:
                ahpDict["locals"]["famil"]["Yes"].append(abs(famLoc))
            elif np.sign(famLoc) > 0:
                ahpDict["locals"]["famil"]["Yes"].append(1/famLoc)
            else:
                ahpDict["locals"]["famil"]["Yes"].append(1.0)
        
            # Genre
            for dex,locs in enumerate(row[34:44]):
                header=df.columns[dex+34].split("-")[1]
                header=re.findall(regexStr, header)
                locs=float(locs)
                if np.sign(locs) > 1:
                    try:
                        ahpDict["locals"]["genre"][header[1]][header[0]].append(locs)
                    except:
                        ahpDict["locals"]["genre"][header[0]][header[1]].append(1/locs)
                elif np.sign(locs) < 0:
                    try:
                        ahpDict["locals"]["genre"][header[0]][header[1]].append(abs(locs))
                    except:
                        ahpDict["locals"]["genre"][header[1]][header[0]].append(1/abs(locs))
                else:
                    try:
                        ahpDict["locals"]["genre"][header[0]][header[1]].append(1.0)
                    except:
                        ahpDict["locals"]["genre"][header[1]][header[0]].append(1.0)
                        
            # Listen
            for dex,locs in enumerate(row[44:47]):
                header=df.columns[dex+44].split("-")[1]
                header=re.findall(regexStr,header)
                locs=float(locs)
                if np.sign(locs) > 1:
                    try:
                        ahpDict["locals"]["listen"][header[1]][header[0]].append(locs)
                    except:
                        ahpDict["locals"]["listen"][header[0]][header[1]].append(1/locs)
                elif np.sign(locs) < 0:
                    try:
                        ahpDict["locals"]["listen"][header[0]][header[1]].append(abs(locs))
                    except:
                        ahpDict["locals"]["listen"][header[1]][header[0]].append(1/abs(locs))
                else:
                    try:
                        ahpDict["locals"]["listen"][header[0]][header[1]].append(1.0)
                    except:
                        ahpDict["locals"]["listen"][header[1]][header[0]].append(1.0)                    
                        
            # Repro
            for dex,locs in enumerate(row[47:50]):
                header=df.columns[dex+47].split("-")[1]
                header=re.findall(regexStr,header)
                locs=float(locs)
                if np.sign(locs) > 1:
                    try:
                        ahpDict["locals"]["repro"][header[1]][header[0]].append(float(locs))
                    except:
                        ahpDict["locals"]["repro"][header[0]][header[1]].append(1/float(locs))
                elif np.sign(locs) < 0:
                    try:
                        ahpDict["locals"]["repro"][header[0]][header[1]].append(float(abs(locs)))
                    except:
                        ahpDict["locals"]["repro"][header[1]][header[0]].append(1/float(locs))                    
                else:
                    try:
                        ahpDict["locals"]["repro"][header[0]][header[1]].append(1.0)
                    except:
                        ahpDict["locals"]["repro"][header[1]][header[0]].append(1.0)                                     
        return ahpDict
        
    def makeAHPMatrix(self,df,globs):
        for g in globs:
            for glob in globs:
                if g == glob:
                    print g,glob,1.0                
                elif g not in df["globals"]:
                    print g, glob, 1/np.mean(df["globals"][glob][glob])
                else:
                    print g, glob, np.mean(df["globals"][g][glob])
        pass
        
a=arayi()
regexStr='[A-Z][a-z]*'
data=a.removeCols("/Users/nus/Documents/GitHub/Singability/data/mTurkResults.csv")

facDict=a.calcFAC(data)

ahpDict=a.calcAHP(data)

globs=["famil","genre","repro","listen"]
# ahpMat=a.makeAHPMatrix(ahpDict,globs)


## Check ahp values
# for key in ahpDict["globals"]:
#     for value in ahpDict["globals"][key]:
#         print key, value, np.mean(ahpDict["globals"][key][value])

# for key in ahpDict["locals"]:
#     if key=="famil":
#         print "local", "famil","Yes","Yes",np.mean(ahpDict["locals"][key]["Yes"])
#         continue
#     for level in ahpDict["locals"][key]:
#         for value in ahpDict["locals"][key][level]:
#             print "local", key,level,value, np.mean(ahpDict["locals"][key][level][value])



SpearmanrResult(correlation=0.47824761203904109, pvalue=0.00044348108357114601)


In [55]:
import operator
sing={}
for song in facDict:
    sing[song]=facDict[song]['sing']
sorted_sing = sorted(sing.items(), key=operator.itemgetter(1),reverse=True)
billR=[]
facR=[]
ahpR=[]
rankDict={}
for idx, ranks in enumerate(sorted_sing):
    rankDict[ranks[0]]={"billR":int(ranks[0].split("_")[0]),"facR":idx}
#     print ",".join(map(str,[idx, ranks[0].split("_")[0]," ".join(ranks[0].split("_")[1:3]),ranks[1]]))
# print sp.spearmanr(billR,facR) 


ahpValues={"locals":{"genre":{"rock":0.227694645,"pop":0.213700581,"country":0.193254532,"alt":0.196807281,"rap":0.168542961},"famil":{"high":0.613463796,"low":0.386536204},"listen":{"low":0.299476454,"mid":0.338568204,"high":0.361955342},"repro":{"easy":0.363546989,"mid":0.333072649,"high":0.303380363}}}
globDic={}
for song in facDict:
    sing[song]=facDict[song]["familiar"]
sorted_sing = sorted(sing.items(), key=operator.itemgetter(1),reverse=True)
for idx, ranks in enumerate(sorted_sing):
    globDic[ranks[0]]={"familiar":None,"genre":None,"listen":None}

    if idx < 26:
        globDic[ranks[0]]["familiar"]=ahpValues["locals"]["famil"]["high"]
        globDic[ranks[0]]["genre"]=ahpValues["locals"]["genre"][ranks[0].split("_")[1]]
    else:
        globDic[ranks[0]]["familiar"]=ahpValues["locals"]["famil"]["low"]
        globDic[ranks[0]]["genre"]=ahpValues["locals"]["genre"][ranks[0].split("_")[1]]
        
for song in facDict:
    sing[song]=facDict[song]["listen"]
sorted_sing = sorted(sing.items(), key=operator.itemgetter(1),reverse=True)

print sorted_sing
for idx, ranks in enumerate(sorted_sing):
    if idx < 18:
        globDic[ranks[0]]["listen"]=ahpValues["locals"]["listen"]["high"]
    elif idx >= 18 and idx > 36:
        globDic[ranks[0]]["listen"]=ahpValues["locals"]["listen"]["mid"]
    else:
        globDic[ranks[0]]["listen"]=ahpValues["locals"]["listen"]["low"]

for song in globDic:
    sing[song]=globDic[song]["genre"]*globDic[song]["listen"]*globDic[song]["familiar"]
sorted_sing = sorted(sing.items(), key=operator.itemgetter(1),reverse=True)


for idx, ranks in enumerate(sorted_sing):
    rankDict[ranks[0]]["ahpR"]=idx
    
for song in rankDict:
    billR.append(rankDict[song]["billR"])
    ahpR.append(rankDict[song]["ahpR"])
    facR.append(rankDict[song]["facR"])

# print rankDict
print sp.spearmanr(ahpR,billR)
print sp.spearmanr(ahpR,facR)
print sp.spearmanr(facR,billR)
    

[('43_pop_male_split.mp3', 112), ('42_pop_male_split.mp3', 111), ('42_rap_female_split.mp3', 106), ('46_alt_female_split.mp3', 104), ('46_country_male_split.mp3', 103), ('41_pop_female_split.mp3', 103), ('33_rock_female_split.mp3', 103), ('43_pop_female_split.mp3', 101), ('49_country_female_split.mp3', 100), ('41_pop_male_split.mp3', 100), ('42_country_female_split.mp3', 98), ('41_rap_female_split.mp3', 95), ('44_pop_female_split.mp3', 94), ('45_country_female_split.mp3', 93), ('46_rap_female_split.mp3', 90), ('46_rap_male_split.mp3', 89), ('50_rap_female_split.mp3', 89), ('42_pop_female_split.mp3', 89), ('41_rock_female_split.mp3', 89), ('48_country_male_split.mp3', 88), ('50_country_male_split.mp3', 87), ('44_country_female_split.mp3', 86), ('48_pop_female_split.mp3', 86), ('42_country_male_split.mp3', 86), ('43_rock_female_split.mp3', 85), ('49_pop_male_split.mp3', 84), ('41_rap_male_split.mp3', 83), ('50_pop_male_split.mp3', 81), ('45_alt_male_split.mp3', 79), ('65_rock_female_spli